In [1]:
import os
import shutil
import subprocess
import tempfile
from IPython.core.magic import line_magic, Magics, magics_class

@magics_class
class CacheMagics(Magics):
    @line_magic
    def cleanup_cache(self, line):
        """Delete all contents of ~/.cache directory"""
        cache_dir = os.path.expanduser('~/.cache')
        if os.path.exists(cache_dir):
            shutil.rmtree(cache_dir)
            os.makedirs(cache_dir)
            print("~/.cache directory cleared!")
        else:
            print("~/.cache directory doesn't exist")

# Register the magic
get_ipython().register_magic_function(CacheMagics().cleanup_cache, 'line', 'cleanup_cache')

In [1]:
#@title Install dependencies. { display-mode: "form" }
# %%capture THIS DOES NOT WORK
%pip -q install sentencepiece
%pip -q install git+https://github.com/huggingface/transformers.git # had a -U before
%pip -q install git+https://github.com/huggingface/peft.git
%pip -q install git+https://github.com/huggingface/accelerate.git
%pip -q install omegaconf pytorch_lightning biopython ml_collections einops py3Dmol
%pip -q install ipywidgets
#!wget -q -nc https://mmseqs.com/foldseek/foldseek-linux-avx2.tar.gz; tar xzf foldseek-linux-avx2.tar.gz; export PATH=$(pwd)/foldseek/bin/:$PATH
!wget -q -nc https://mmseqs.com/foldseek/foldseek-osx-universal.tar.gz; tar xvzf foldseek-osx-universal.tar.gz; export PATH=$(pwd)/foldseek/bin/:$PATH

Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.57.0.dev0 requires huggingface-hub<1.0,>=0.34.0, but you have huggingface-hub 1.0.0rc2 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
x foldseek/
x foldseek/README.md
x foldseek/bin/
x foldseek/bin/foldseek


In [2]:
#@title Import dependencies. { display-mode: "form" }
from transformers import T5Tokenizer, AutoModelForSeq2SeqLM
import torch    
if torch.cuda.is_available():
  device = torch.device("cuda")
  print("Using CUDA")
elif torch.backends.mps.is_available():
  device = torch.device("mps")
  print("Using MPS")
elif torch.backends.opencl.is_available():  # Example for an additional device (e.g., OpenCL)
  device = torch.device("opencl")
  print("Using OpenCL")
else:
  device = torch.device("cpu")
  print("Using CPU")
if not torch.cuda.is_available():
  print("Warning: You are running this notebook without GPU which will be slow.")

/Users/michael.heinzinger/scripts/schweden_tutorial/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Using MPS


In [3]:
#@title Load ProstT5. { display-mode: "form" }
# Load the tokenizer
tokenizer = T5Tokenizer.from_pretrained('Rostlab/ProstT5_fp16', do_lower_case=False, legacy=True)

# Load the model
model = AutoModelForSeq2SeqLM.from_pretrained("Rostlab/ProstT5_fp16",low_cpu_mem_usage=True, device_map="auto", use_safetensors=False, dtype=torch.float16)
model = model.eval()
model = model.half()

In [4]:
#@title Example: how to get 3Di from PDB/AFDB
query_ID = 'A0A6G0XC32' #@param {type:"string"}
!mkdir -p $query_ID
!wget -q -O $query_ID/AF-$query_ID-F1-model_v4.pdb https://alphafold.ebi.ac.uk/files/AF-$query_ID-F1-model_v4.pdb
!foldseek/bin/foldseek createdb $query_ID/ $query_ID/queryDB
!foldseek/bin/foldseek lndb $query_ID/queryDB_h $query_ID/queryDB_ss_h
!foldseek/bin/foldseek convert2fasta $query_ID/queryDB_ss $query_ID/queryDB_ss.fasta

createdb A0A6G0XC32/ A0A6G0XC32/queryDB 

MMseqs Version:             	8979d230fb64c7089380b652758d8705493ed4a5
Use GPU                     	0
Path to ProstT5             	
Chain name mode             	0
Model name mode             	0
Createdb extraction mode    	0
Interface distance threshold	8
Write mapping file          	0
Mask b-factor threshold     	0
Coord store mode            	2
Write lookup file           	1
Input format                	0
File Inclusion Regex        	.*
File Exclusion Regex        	^$
Threads                     	16
Verbosity                   	3

Output file: A0A6G0XC32/queryDB
[=================================================================] 100.00% 1 eta -
Time for merging to queryDB_ss: 0h 0m 0s 3ms
Time for merging to queryDB_h: 0h 0m 0s 3ms
Time for merging to queryDB_ca: 0h 0m 0s 3ms
Time for merging to queryDB: 0h 0m 0s 3ms
Ignore 0 out of 1.
Too short: 0, incorrect: 0, not proteins: 0.
Time for processing: 0h 0m 0s 24ms
lndb A0A6G0XC32/queryDB_h A0A

In [ ]:
#%cleanup_cache

~/.cache directory cleared!


In [5]:
#@title Read in file in FASTA format. { display-mode: "form" }
def read_fasta( in_path, is_3Di ):
    '''
        Reads in fasta file containing a single or multiple sequences.
        Returns dictionary.
    '''

    sequences = dict()
    with open( in_path, 'r' ) as fasta_f:
        for line in fasta_f:
            # get uniprot ID from header and create new entry
            if line.startswith('>'):
                uniprot_id = line.split(" ")[0].replace('>', '').replace(".pdb","").strip()
                sequences[ uniprot_id ] = ''
            else:
                # repl. all whie-space chars and join seqs spanning multiple lines
                if is_3Di:
                    sequences[ uniprot_id ] += ''.join( line.split() ).replace("-","").lower() # drop gaps and cast to lower-case
                else:
                    sequences[ uniprot_id ] += ''.join( line.split() ).replace("-","")

    example = sequences[uniprot_id]

    print("##########################")
    print(f"Input is 3Di: {is_3Di}")
    print(f"Example sequence: >{uniprot_id}\n{example}")
    print("##########################")

    return sequences

In [ ]:
# this function converts raw ESMFold output to PDB file
from transformers.models.esm.openfold_utils.protein import to_pdb, Protein as OFProtein
from transformers.models.esm.openfold_utils.feats import atom14_to_atom37

def convert_outputs_to_pdb(outputs):
    final_atom_positions = atom14_to_atom37(outputs["positions"][-1], outputs)
    outputs = {k: v.to("cpu").numpy() for k, v in outputs.items()}
    final_atom_positions = final_atom_positions.cpu().numpy()
    final_atom_mask = outputs["atom37_atom_exists"]
    pdbs = []
    for i in range(outputs["aatype"].shape[0]):
        aa = outputs["aatype"][i]
        pred_pos = final_atom_positions[i]
        mask = final_atom_mask[i]
        resid = outputs["residue_index"][i] + 1
        pred = OFProtein(
            aatype=aa,
            atom_positions=pred_pos,
            atom_mask=mask,
            residue_index=resid,
            b_factors=outputs["plddt"][i],
            chain_index=outputs["chain_index"][i] if "chain_index" in outputs else None,
        )
        pdbs.append(to_pdb(pred))
    return pdbs

In [8]:
# Alternatively, let's run the local ESMFold version directly from huggingface in case the API is not working properly
# Here some very basic example script to test whether the local version is working:
import torch
from transformers import AutoTokenizer, EsmForProteinFolding

print("mps available:", torch.backends.mps.is_available())
esmfold_model = EsmForProteinFolding.from_pretrained("facebook/esmfold_v1").eval()
esmfold_tok = AutoTokenizer.from_pretrained("facebook/esmfold_v1")
esmfold_model.esm = esmfold_model.esm.half()
torch.backends.cuda.matmul.allow_tf32 = True
esmfold_model.trunk.set_chunk_size(64)
inputs = esmfold_tok(["MLKNVQVQLV"], return_tensors="pt", add_special_tokens=False)
with torch.no_grad():
    out = esmfold_model(**inputs)
print(out.positions.shape)  # (batch, residues, atoms, coord)

mps available: True


Some weights of EsmForProteinFolding were not initialized from the model checkpoint at facebook/esmfold_v1 and are newly initialized: ['esm.contact_head.regression.bias', 'esm.contact_head.regression.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


torch.Size([8, 1, 10, 14, 3])


In [ ]:
# this part is only needed in case the ESMFold API did not work. 
# we will rely on the local hugginface version described here:
# https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/protein_folding.ipynb#scrollTo=6f8ba985
import locale
import time
import requests
locale.getpreferredencoding = lambda: "UTF-8"
!mkdir -p $query_ID/gen_seqs

seq_dict = read_fasta(f"{query_ID}/queryDB_ss.fasta",is_3Di=True)

gen_kwargs =  {
            "do_sample": True,
            "top_p" : 0.85,
            "temperature" : 1.0,
            "top_k" : 3,
            "repetition_penalty" : 1.2,
            }

generated_sequences=dict()
for seq_idx, (fasta_id, seq) in enumerate(seq_dict.items(),1): # for each sequence in the FASTA file
    seq_len = len(seq)
    seq = seq.replace('U','X').replace('Z','X').replace('O','X').replace("B","X")
    seq = " ".join(list(seq))

    max_len=seq_len+1 # ensures that the generated proteins are not shorter/longer
    min_len=seq_len+1 # should not be necessary as the model was only trained on pairs of sequences/structures with identical length

    # starting point tokens
    start_encoding = tokenizer.batch_encode_plus( ["<fold2AA>" + " " + seq],
                                       add_special_tokens=True,
                                       return_tensors='pt'
                                       ).to(device)
    
    print(f"3Di sequence as fed to the model (white-space seperated AAs:\n{seq}")

    with torch.no_grad():
      # forward translation tokens
      target = model.generate(
                            start_encoding.input_ids,
                            attention_mask=start_encoding.attention_mask,
                            max_length=max_len, # max length of generated text
                            min_length=min_len, # minimum length of the generated text
                            length_penalty=1.0, # import for correct normalization of scores
                            num_return_sequences=10, # return a total of 10 sequences
                            **gen_kwargs
                            )
    t_strings = tokenizer.batch_decode( target, skip_special_tokens=True )
    for gen_seq_idx, t in enumerate(t_strings,0):
      gen_seq = "".join( t.split(" "))
      gen_seq_id = fasta_id + f"_{gen_seq_idx}"
      inputs = esmfold_tok([gen_seq], return_tensors="pt", add_special_tokens=False)
      with torch.no_grad():
        out = esmfold_model(**inputs)
      pdb = convert_outputs_to_pdb(out)

      with open(f"{query_ID}/gen_seqs/{gen_seq_id}.pdb","w") as out_f:
        out_f.write("".join(pdb))
      print(f"Successfully folded {gen_seq}")

##########################
Input is 3Di: True
Example sequence: >AF-A0A6G0XC32-F1-model_v4
ddfdaedepacccpdqedadagarhqyyhhanvqnhaadaynyqnhqyyepenhanhqlvrhlvnhdhqnhayyehalhhhpdddplvsllvnlqshqnhqhyehalpqaelssvvsclpsnpnhqeyeyedapprpphhhhddpvsvvvscvvvvshdydyd
##########################
3Di sequence as fed to the model (white-space seperated AAs:
d d f d a e d e p a c c c p d q e d a d a g a r h q y y h h a n v q n h a a d a y n y q n h q y y e p e n h a n h q l v r h l v n h d h q n h a y y e h a l h h h p d d d p l v s l l v n l q s h q n h q h y e h a l p q a e l s s v v s c l p s n p n h q e y e y e d a p p r p p h h h h d d p v s v v v s c v v v v s h d y d y d
torch.Size([8, 1, 163, 14, 3])
Successfully folded MGGLKSLNVARCDQITELHITSKSLEELNLWECYSLQKFTLNCPNLSKLSLSFCSQIDVEELISSLSCPNLSHLDLSGCFPKGESIDQLVSKFFSSLPNITHLSLSGTNLSEKTLHAVLDSCLHLELLNVQRNSSSSRYRVICSEKLLEDFRAERPNVKVVFS
torch.Size([8, 1, 163, 14, 3])
Successfully folded MSGLQKLDVSRNEYITELCIAHPNLTFLDLSWCSSLKRLSLNCPKLEELDLSFCSKLEIDEFFEN

In [ ]:
#%cleanup_cache

UsageError: Line magic function `%cleanup_cache` not found.


In [5]:
import Bio.PDB
from Bio.Data import IUPACData
from Bio import pairwise2
from Bio.Align import substitution_matrices


/Users/michael.heinzinger/scripts/schweden_tutorial/.venv/lib/python3.9/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


In [6]:
def seq_identity_similarity(
    seq1: str,
    seq2: str,
    matrix=substitution_matrices.load("BLOSUM62"),
    gap_open: float = -10.0, # values taken from https://biopython.org/docs/latest/Tutorial/chapter_pairwise2.html
    gap_extend: float = -0.5,
):
    """
    Global alignment with a substitution matrix.
    Returns (percent_identity, percent_similarity, (aln1, aln2), raw_score)
    - identity: exact matches / aligned (non-gap) positions
    - similarity: BLOSUM62 "positive" pairs (score > 0) / aligned (non-gap) positions
    """
    # One best global alignment scored by the matrix
    aln = pairwise2.align.globalds(seq1, seq2, matrix,
                                   open=gap_open, extend=gap_extend,
                                   one_alignment_only=True)[0]
    aln1, aln2, score, _, _ = aln

    # count over positions where both are residues (ignore gaps in either)
    aligned_positions = 0
    matches = 0
    positives = 0

    for a, b in zip(aln1, aln2):
        if a == "-" or b == "-":
            continue
        aligned_positions += 1

        if a == b:
            matches += 1

        # look up (a,b) in the (symmetric) matrix
        key = (a, b) if (a, b) in matrix else (b, a)
        sub = matrix.get(key, None)
        if sub is not None and sub > 0:
            positives += 1

    pid = (matches / aligned_positions) if aligned_positions else 0.0
    psim = (positives / aligned_positions) if aligned_positions else 0.0
    return pid, psim, (aln1, aln2), score, aln

In [ ]:
#@title Compute RMSD between generated sequences (ESMFold) and groundtruth (AFDB). { display-mode: "form" }
# https://colab.research.google.com/github/pb3lab/ibm3202/blob/master/tutorials/lab02_molviz.ipynb
from pathlib import Path
three_to_one = IUPACData.protein_letters_3to1
# Start the parsers
pdb_parser = Bio.PDB.PDBParser(QUIET = True) # structure parser
seq_parser = Bio.PDB.PPBuilder() # sequence parser

# Get the structures
ref_structure = pdb_parser.get_structure("reference", f"{query_ID}/AF-{query_ID}-F1-model_v4.pdb")
best_RMSD = None
best_alignment = None
for p in Path(f"{query_ID}/gen_seqs").rglob("*.pdb"):
  sample_structure = pdb_parser.get_structure("sample", p)

  # Use the first model in the pdb-files for alignment
  # Change the number 0 if you want to align to another structure
  ref_model    = ref_structure[0]
  sample_model = sample_structure[0]

  # Make a list of the atoms (in the structures) you wish to align.
  # In this case we use CA atoms whose index is in the specified range
  ref_atoms = []
  sample_atoms = []

  # Make a list of amino acids
  ref_seq = []
  sample_seq = []

  # Iterate of all chains in the model in order to find all residues
  for ref_chain in ref_model:
    # Iterate of all residues in each model in order to find proper atoms
    for ref_res in ref_chain:
        # Append CA atom to list
        ref_atoms.append(ref_res['CA'])
        ref_seq.append(three_to_one[ref_res.get_resname().capitalize()])
  ref_seq = "".join(ref_seq)

  # Do the same for the sample structure
  for sample_chain in sample_model:
    for sample_res in sample_chain:
        sample_atoms.append(sample_res['CA'])
        sample_seq.append(three_to_one[sample_res.get_resname().capitalize()])
  sample_seq = "".join(sample_seq)
  assert len(sample_seq)==len(ref_seq), print(len(sample_seq),len(ref_seq))
  # Now we initiate the superimposer:
  super_imposer = Bio.PDB.Superimposer()
  super_imposer.set_atoms(ref_atoms, sample_atoms)
  super_imposer.apply(sample_model.get_atoms())

  # Compute sequence identity/similarity:
  pid, psim, (aln1, aln2), score, aln = seq_identity_similarity(ref_seq, sample_seq)

  # Print RMSD:
  print(f'The calculated RMSD for {p} is: {super_imposer.rms:.2f}Å at {pid:.2f}/{psim:.2f} identity/similarity')
  if best_RMSD is None or best_RMSD > super_imposer.rms:
    best_RMSD = super_imposer.rms
    best_alignment = aln
    # Save the aligned version
    print("Saving better structure.")
    io = Bio.PDB.PDBIO()
    io.set_structure(sample_structure)
    io.save(f"{query_ID}/aligned.pdb")

print(f"Best RMSD is {best_RMSD} with the following alignment:")
print(pairwise2.format_alignment(*best_alignment))

The calculated RMSD for A0A6G0XC32/gen_seqs/AF-A0A6G0XC32-F1-model_v4_8.pdb is: 2.91Å at 0.31/0.55 identity/similarity
Saving better structure.
The calculated RMSD for A0A6G0XC32/gen_seqs/AF-A0A6G0XC32-F1-model_v4_9.pdb is: 2.06Å at 0.35/0.54 identity/similarity
Saving better structure.
The calculated RMSD for A0A6G0XC32/gen_seqs/AF-A0A6G0XC32-F1-model_v4_4.pdb is: 2.32Å at 0.30/0.51 identity/similarity
The calculated RMSD for A0A6G0XC32/gen_seqs/AF-A0A6G0XC32-F1-model_v4_5.pdb is: 1.95Å at 0.35/0.58 identity/similarity
Saving better structure.
The calculated RMSD for A0A6G0XC32/gen_seqs/AF-A0A6G0XC32-F1-model_v4_7.pdb is: 2.22Å at 0.35/0.55 identity/similarity
The calculated RMSD for A0A6G0XC32/gen_seqs/AF-A0A6G0XC32-F1-model_v4_6.pdb is: 2.30Å at 0.29/0.48 identity/similarity
The calculated RMSD for A0A6G0XC32/gen_seqs/AF-A0A6G0XC32-F1-model_v4_2.pdb is: 2.04Å at 0.29/0.50 identity/similarity
The calculated RMSD for A0A6G0XC32/gen_seqs/AF-A0A6G0XC32-F1-model_v4_3.pdb is: 2.14Å at 0.3

In [9]:
#@title Display superposition of generated sequence with lowest RMSD. { display-mode: "form" }
# https://colab.research.google.com/github/pb3lab/ibm3202/blob/master/tutorials/lab02_molviz.ipynb
import py3Dmol
view=py3Dmol.view()
view.addModel(open(f"{query_ID}/AF-{query_ID}-F1-model_v4.pdb", 'r').read(),'pdb')
view.addModel(open(f'{query_ID}/aligned.pdb', 'r').read(),'pdb')
view.zoomTo()
view.setBackgroundColor('white')
view.setStyle({'model':-1},{'cartoon': {'color':'green'}})
view.setStyle({'model':-2},{'cartoon': {'color':'red'}})
view.show()
print("Reference structure (AFDB) shown in green; ESMFold prediction of generated sequence shown in red.")
     

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

Reference structure (AFDB) shown in green; ESMFold prediction of generated sequence shown in red.
